# Chapter 8-4: 미니프로젝트 - 날씨 정보 앱

## 프로젝트 개요
- 외부 API를 활용한 날씨 정보 조회 앱
- 모듈과 패키지를 활용한 구조화된 코드
- 내장 모듈만으로 HTTP 요청 처리
- 사용자 친화적 인터페이스 구현

## 1. 프로젝트 구조 설계

In [ ]:
import os
import json
import urllib.request
import urllib.parse
from datetime import datetime

# 프로젝트 구조 생성
project_structure = {
    'weather_app': {
        '__init__.py': '',
        'api_client.py': '',
        'weather_data.py': '',
        'formatter.py': '',
        'config.py': '',
        'main.py': ''
    },
    'data': {},
    'logs': {}
}

def create_structure(structure, base_path='.'):
    for name, content in structure.items():
        path = os.path.join(base_path, name)
        if isinstance(content, dict):
            os.makedirs(path, exist_ok=True)
            create_structure(content, path)
        else:
            with open(path, 'w', encoding='utf-8') as f:
                f.write(content)

create_structure(project_structure)
print("프로젝트 구조가 생성되었습니다!")

## 2. 핵심 모듈 구현

In [ ]:
# weather_app/config.py - 설정 관리
config_code = '''
"""날씨 앱 설정 모듈"""

import os

# API 설정 (실제 사용시 API 키 필요)
WEATHER_API_URL = "http://api.openweathermap.org/data/2.5/weather"
FORECAST_API_URL = "http://api.openweathermap.org/data/2.5/forecast"
API_KEY = os.environ.get('WEATHER_API_KEY', 'demo_key')

# 기본 설정
DEFAULT_CITY = "Seoul"
DEFAULT_UNITS = "metric"  # celsius
CACHE_DURATION = 300  # 5분

# 파일 경로
DATA_DIR = "data"
LOG_DIR = "logs"
CACHE_FILE = os.path.join(DATA_DIR, "weather_cache.json")
LOG_FILE = os.path.join(LOG_DIR, "weather_app.log")

# 지원 도시 목록
SUPPORTED_CITIES = [
    "Seoul", "Busan", "Incheon", "Daegu", "Daejeon",
    "Gwangju", "Ulsan", "Jeju", "Tokyo", "Beijing",
    "New York", "London", "Paris", "Sydney"
]
'''

with open('weather_app/config.py', 'w', encoding='utf-8') as f:
    f.write(config_code)

print("config.py 모듈 작성 완료!")

In [ ]:
# weather_app/api_client.py - API 클라이언트
api_client_code = '''
"""날씨 API 클라이언트 모듈"""

import urllib.request
import urllib.parse
import json
import time
from . import config

class WeatherAPIClient:
    """날씨 API 클라이언트"""
    
    def __init__(self, api_key=None):
        self.api_key = api_key or config.API_KEY
        self.base_url = config.WEATHER_API_URL
        self.cache = {}
    
    def _make_request(self, url):
        """HTTP 요청 수행"""
        try:
            req = urllib.request.Request(url)
            req.add_header('User-Agent', 'Weather-App/1.0')
            
            with urllib.request.urlopen(req, timeout=10) as response:
                return json.loads(response.read().decode())
        except Exception as e:
            return {"error": str(e)}
    
    def get_weather(self, city, units="metric"):
        """도시별 현재 날씨 조회"""
        cache_key = f"{city}_{units}"
        
        # 캐시 확인
        if cache_key in self.cache:
            cached_data, timestamp = self.cache[cache_key]
            if time.time() - timestamp < config.CACHE_DURATION:
                return cached_data
        
        # API 호출
        params = {
            'q': city,
            'appid': self.api_key,
            'units': units
        }
        
        url = f"{self.base_url}?{urllib.parse.urlencode(params)}"
        data = self._make_request(url)
        
        # 성공시 캐시 저장
        if "error" not in data:
            self.cache[cache_key] = (data, time.time())
        
        return data
    
    def get_mock_weather(self, city):
        """모의 날씨 데이터 (API 키 없을 때)"""
        import random
        
        mock_data = {
            "name": city,
            "main": {
                "temp": round(random.uniform(-10, 35), 1),
                "humidity": random.randint(30, 90),
                "pressure": random.randint(980, 1030)
            },
            "weather": [{
                "main": random.choice(["Clear", "Clouds", "Rain", "Snow"]),
                "description": random.choice(["clear sky", "few clouds", "light rain", "snow"])
            }],
            "wind": {
                "speed": round(random.uniform(0, 20), 1)
            },
            "sys": {
                "country": "KR" if city in ["Seoul", "Busan"] else "XX"
            }
        }
        
        return mock_data
'''

with open('weather_app/api_client.py', 'w', encoding='utf-8') as f:
    f.write(api_client_code)

print("api_client.py 모듈 작성 완료!")

In [ ]:
# weather_app/main.py - 메인 애플리케이션
main_code = '''
"""날씨 정보 앱 메인 모듈"""

import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

from weather_app.api_client import WeatherAPIClient
from weather_app import config
from datetime import datetime

class WeatherApp:
    """날씨 정보 앱"""
    
    def __init__(self):
        self.client = WeatherAPIClient()
        self.running = True
    
    def display_menu(self):
        """메뉴 출력"""
        print("\n" + "="*50)
        print("🌤️  날씨 정보 앱")
        print("="*50)
        print("1. 현재 날씨 조회")
        print("2. 여러 도시 날씨 비교")
        print("3. 지원 도시 목록")
        print("4. 앱 정보")
        print("0. 종료")
        print("="*50)
    
    def get_weather_info(self, city):
        """날씨 정보 조회 및 포맷팅"""
        data = self.client.get_mock_weather(city)  # 모의 데이터 사용
        
        if "error" in data:
            return f"❌ 오류: {data['error']}"
        
        weather_info = f"""
📍 {data['name']} ({data.get('sys', {}).get('country', 'XX')})
🌡️  온도: {data['main']['temp']}°C
☁️  날씨: {data['weather'][0]['main']} ({data['weather'][0]['description']})
💧 습도: {data['main']['humidity']}%
🌬️  풍속: {data['wind']['speed']} m/s
🔽 기압: {data['main']['pressure']} hPa
⏰ 조회시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
"""
        return weather_info
    
    def compare_cities_weather(self):
        """여러 도시 날씨 비교"""
        cities = input("비교할 도시들을 쉼표로 구분해서 입력하세요: ").strip().split(',')
        cities = [city.strip() for city in cities if city.strip()]
        
        if len(cities) < 2:
            print("❌ 최소 2개 도시를 입력해주세요.")
            return
        
        print("\n" + "="*60)
        print("🌍 도시별 날씨 비교")
        print("="*60)
        
        weather_data = []
        for city in cities:
            data = self.client.get_mock_weather(city)
            if "error" not in data:
                weather_data.append({
                    'city': data['name'],
                    'temp': data['main']['temp'],
                    'weather': data['weather'][0]['main'],
                    'humidity': data['main']['humidity']
                })
        
        # 온도순 정렬
        weather_data.sort(key=lambda x: x['temp'], reverse=True)
        
        for i, data in enumerate(weather_data, 1):
            emoji = "🔥" if i == 1 else "❄️" if i == len(weather_data) else "🌡️"
            print(f"{emoji} {i}. {data['city']}: {data['temp']}°C, {data['weather']}, 습도 {data['humidity']}%")
    
    def run(self):
        """앱 실행"""
        print("날씨 정보 앱을 시작합니다!")
        
        while self.running:
            try:
                self.display_menu()
                choice = input("\n선택하세요 (0-4): ").strip()
                
                if choice == "1":
                    city = input("도시 이름을 입력하세요: ").strip()
                    if city:
                        print(self.get_weather_info(city))
                    else:
                        print("❌ 도시 이름을 입력해주세요.")
                
                elif choice == "2":
                    self.compare_cities_weather()
                
                elif choice == "3":
                    print("\n📋 지원 도시 목록:")
                    for i, city in enumerate(config.SUPPORTED_CITIES, 1):
                        print(f"{i:2d}. {city}")
                
                elif choice == "4":
                    print("\n📱 앱 정보")
                    print(f"   버전: 1.0.0")
                    print(f"   개발: Python 학습 프로젝트")
                    print(f"   기능: 날씨 정보 조회 및 비교")
                
                elif choice == "0":
                    print("\n👋 날씨 앱을 종료합니다. 안녕히 가세요!")
                    self.running = False
                
                else:
                    print("❌ 올바른 번호를 선택해주세요.")
                
                if self.running:
                    input("\n계속하려면 Enter를 누르세요...")
            
            except KeyboardInterrupt:
                print("\n\n👋 프로그램을 종료합니다.")
                break
            except Exception as e:
                print(f"\n❌ 오류가 발생했습니다: {e}")

if __name__ == "__main__":
    app = WeatherApp()
    app.run()
'''

with open('weather_app/main.py', 'w', encoding='utf-8') as f:
    f.write(main_code)

print("main.py 메인 애플리케이션 작성 완료!")

## 3. 프로젝트 실행 테스트

In [ ]:
# __init__.py 파일 작성
init_code = '''
"""날씨 정보 앱 패키지"""

__version__ = "1.0.0"
__author__ = "Python Learning"
__description__ = "모듈과 패키지를 활용한 날씨 정보 앱"

from .api_client import WeatherAPIClient
from . import config
'''

with open('weather_app/__init__.py', 'w', encoding='utf-8') as f:
    f.write(init_code)

# 간단한 테스트 실행
print("=== 날씨 앱 테스트 ===")

# API 클라이언트 테스트
import sys
sys.path.append('.')

from weather_app.api_client import WeatherAPIClient

client = WeatherAPIClient()
test_cities = ["Seoul", "Tokyo", "New York"]

print("🌤️ 테스트 도시들의 모의 날씨 데이터:")
for city in test_cities:
    data = client.get_mock_weather(city)
    print(f"\n📍 {data['name']}:")
    print(f"   온도: {data['main']['temp']}°C")
    print(f"   날씨: {data['weather'][0]['main']}")
    print(f"   습도: {data['main']['humidity']}%")

print("\n✅ 날씨 앱 패키지가 성공적으로 생성되었습니다!")
print("\n실행 방법:")
print("python weather_app/main.py")

## 4. 프로젝트 완성 및 정리

### 🎯 구현된 기능

1. **모듈화된 구조**
   - config.py: 설정 관리
   - api_client.py: API 통신
   - main.py: 메인 애플리케이션

2. **주요 기능**
   - 현재 날씨 조회
   - 여러 도시 날씨 비교
   - 캐싱 시스템
   - 사용자 친화적 인터페이스

3. **활용된 개념**
   - 패키지와 모듈 구조
   - 내장 모듈 활용 (urllib, json)
   - 클래스와 객체지향 프로그래밍
   - 예외 처리

### 💡 확장 가능한 기능

- 실제 날씨 API 연동
- 데이터베이스 저장
- 웹 인터페이스 추가
- 알림 기능
- 차트 및 그래프

### 🏆 학습 성과

Chapter 8을 통해 모듈과 패키지의 중요성과 활용법을 실제 프로젝트를 통해 경험했습니다!

다음 Chapter에서는 **파일 입출력**에 대해 학습하겠습니다!